# Text Generation with Variations of Markov Chains and Evaluating Text Quality with BLEU Scores

## Objective
This project is presented as a story, starting with text generation using a simple Markov Chain and evaluating its quality. The evaluation is performed by comparing the generated text to a reference text using **BLEU scores**. BLEU is a widely used metric for assessing the quality of machine-generated text, measuring its similarity to a reference text.

### Progression of Methods
After reviewing the initial BLEU scores, I will aim to improve the scores through transformations of the Markov Chain approach. The steps include:
1. **First Markov Chain:**
2. **First Markov Chain with a Fallback Method:** .
3. **Second Markov Chain:** Enhancing the model with improved n-gram handling.
4. **Second Markov Chain with a Fallback Method:**
5. **Weighted Random Selection:** Incorporating weighted probabilities, with **fallbacks to bigrams and unigrams**.
6. **Final Version:** Eliminating the fallback to unigrams and instead selecting words from the reference text.

### Results
This project is intended as a demonstration of various text generation and evaluation techniques, rather than a showcase of high-quality generated text. The results are naturally limited by the simplicity of the methods and the relatively small dataset used, especially when compared to the extensive datasets and advanced architectures of modern large language models (LLMs) like ChatGPT. Consequently, the generated text is not expected to match the fluency or coherence of those models.


### Data Source
The text data used in this project comprises all available books from the *Anne of Green Gables* series by Lucy Maud Montgomery, downloaded from [Project Gutenberg](https://www.gutenberg.org/). Project Gutenberg is an online library of free eBooks.

The books were combined into a single TXT file. Since this project is developed on the Databricks platform, the file was converted into a table within the Databricks Catalog.



##Installations
Install all required libraries

In [0]:
!pip install --upgrade pip -q
!pip install nltk -q

In [0]:
import random
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import numpy as np
import re

In [0]:
book_rdd = sc.textFile('/FileStore/tables/green_gables.txt')
book_rdd.take(5)

Out[3]: ['CHAPTER I.',
 'Mrs. Rachel Lynde Is Surprised',
 'MRS. Rachel Lynde lived just where the Avonlea main road dipped down into a little hollow, fringed with alders and ladies’ eardrops and traversed by a brook that had its source away back in the woods of the old Cuthbert place; it was reputed to be an intricate, headlong brook in its earlier course through those woods, with dark secrets of pool and cascade; but by the time it reached Lynde’s Hollow it was a quiet, well-conducted little stream, for not even a brook could run past Mrs. Rachel Lynde’s door without due regard for decency and decorum; it probably was conscious that Mrs. Rachel was sitting at her window, keeping a sharp eye on everything that passed, from brooks and children up, and that if she noticed anything odd or out of place she would never rest until she had ferreted out the whys and wherefores thereof.',
 '',
 'There are plenty of people, in Avonlea and out of it, who can attend closely to their neighbors’ bu

My dataset contains numerous quotation marks, which can lead to issues during text generation. For example, a quotation might start but never properly close, as the model doesn't recognize that the quotation was opened. To address this, the data requires preprocessing. I will remove all quotation marks, along with punctuation such as periods (`.`), question marks (`?`), exclamation marks (`!`), semicolons (`;`), and hyphens (`-`).

In [0]:

# Function to clean text
def clean_text(line):
    line = re.sub(r"[\"“”]", "", line)
    line = re.sub(r"[.!?]", "", line)
    line = re.sub(r";", "", line)
    line = re.sub(r"-", " ", line)
    return line

cleaned_rdd = book_rdd.map(clean_text)
cleaned_rdd.take(5)

Out[4]: ['CHAPTER I',
 'Mrs Rachel Lynde Is Surprised',
 'MRS Rachel Lynde lived just where the Avonlea main road dipped down into a little hollow, fringed with alders and ladies’ eardrops and traversed by a brook that had its source away back in the woods of the old Cuthbert place it was reputed to be an intricate, headlong brook in its earlier course through those woods, with dark secrets of pool and cascade but by the time it reached Lynde’s Hollow it was a quiet, well conducted little stream, for not even a brook could run past Mrs Rachel Lynde’s door without due regard for decency and decorum it probably was conscious that Mrs Rachel was sitting at her window, keeping a sharp eye on everything that passed, from brooks and children up, and that if she noticed anything odd or out of place she would never rest until she had ferreted out the whys and wherefores thereof',
 '',
 'There are plenty of people, in Avonlea and out of it, who can attend closely to their neighbors’ business by

The same preprocessing logic must also be applied to the reference text to ensure a more accurate calculation of the BLEU score, as it depends on the overlaps between the candidate (generated) text and the reference text. 

**The reference text** is one of the chapters taken from the book. By comparing the reference text with the candidate text, we can assess how well the generated text mimics the writing style of Lucy Maud Montgomery.

Since the reference text is loaded as a string, the preprocessing function will look a little bit different. 

In [0]:
reference_text = """
“So you had tea at the stone house with Lavendar Lewis?” said Marilla at the breakfast table next morning. “What is she like now? It’s over fifteen years since I saw her last . . . it was one Sunday in Grafton church. I suppose she has changed a great deal. Davy Keith, when you want something you can’t reach, ask to have it passed and don’t spread yourself over the table in that fashion. Did you ever see Paul Irving doing that when he was here to meals?”

“But Paul’s arms are longer’n mine,” brumbled Davy. “They’ve had eleven years to grow and mine’ve only had seven. ‘Sides, I did ask, but you and Anne was so busy talking you didn’t pay any ‘tention. ‘Sides, Paul’s never been here to any meal escept tea, and it’s easier to be p’lite at tea than at breakfast. You ain’t half as hungry. It’s an awful long while between supper and breakfast. Now, Anne, that spoonful ain’t any bigger than it was last year and I’m ever so much bigger.”

“Of course, I don’t know what Miss Lavendar used to look like but I don’t fancy somehow that she has changed a great deal,” said Anne, after she had helped Davy to maple syrup, giving him two spoonfuls to pacify him. “Her hair is snow-white but her face is fresh and almost girlish, and she has the sweetest brown eyes . . . such a pretty shade of wood-brown with little golden glints in them . . . and her voice makes you think of white satin and tinkling water and fairy bells all mixed up together.”

“She was reckoned a great beauty when she was a girl,” said Marilla. “I never knew her very well but I liked her as far as I did know her. Some folks thought her peculiar even then. Davy, if ever I catch you at such a trick again you’ll be made to wait for your meals till everyone else is done, like the French.”

Most conversations between Anne and Marilla in the presence of the twins, were punctuated by these rebukes Davy-ward. In this instance, Davy, sad to relate, not being able to scoop up the last drops of his syrup with his spoon, had solved the difficulty by lifting his plate in both hands and applying his small pink tongue to it. Anne looked at him with such horrified eyes that the little sinner turned red and said, half shamefacedly, half defiantly,

“There ain’t any wasted that way.”

“People who are different from other people are always called peculiar,” said Anne. “And Miss Lavendar is certainly different, though it’s hard to say just where the difference comes in. Perhaps it is because she is one of those people who never grow old.”

“One might as well grow old when all your generation do,” said Marilla, rather reckless of her pronouns. “If you don’t, you don’t fit in anywhere. Far as I can learn Lavendar Lewis has just dropped out of everything. She’s lived in that out of the way place until everybody has forgotten her. That stone house is one of the oldest on the Island. Old Mr. Lewis built it eighty years ago when he came out from England. Davy, stop joggling Dora’s elbow. Oh, I saw you! You needn’t try to look innocent. What does make you behave so this morning?”

“Maybe I got out of the wrong side of the bed,” suggested Davy. “Milty Boulter says if you do that things are bound to go wrong with you all day. His grandmother told him. But which is the right side? And what are you to do when your bed’s against the wall? I want to know.”

“I’ve always wondered what went wrong between Stephen Irving and Lavendar Lewis,” continued Marilla, ignoring Davy. “They were certainly engaged twenty-five years ago and then all at once it was broken off. I don’t know what the trouble was but it must have been something terrible, for he went away to the States and never come home since.”

“Perhaps it was nothing very dreadful after all. I think the little things in life often make more trouble than the big things,” said Anne, with one of those flashes of insight which experience could not have bettered. “Marilla, please don’t say anything about my being at Miss Lavendar’s to Mrs. Lynde. She’d be sure to ask a hundred questions and somehow I wouldn’t like it . . . nor Miss Lavendar either if she knew, I feel sure.”

“I daresay Rachel would be curious,” admitted Marilla, “though she hasn’t as much time as she used to have for looking after other people’s affairs. She’s tied home now on account of Thomas; and she’s feeling pretty downhearted, for I think she’s beginning to lose hope of his ever getting better. Rachel will be left pretty lonely if anything happens to him, with all her children settled out west, except Eliza in town; and she doesn’t like her husband.”

Marilla’s pronouns slandered Eliza, who was very fond of her husband.

“Rachel says if he’d only brace up and exert his will power he’d get better. But what is the use of asking a jellyfish to sit up straight?” continued Marilla. “Thomas Lynde never had any will power to exert. His mother ruled him till he married and then Rachel carried it on. It’s a wonder he dared to get sick without asking her permission. But there, I shouldn’t talk so. Rachel has been a good wife to him. He’d never have amounted to anything without her, that’s certain. He was born to be ruled; and it’s well he fell into the hands of a clever, capable manager like Rachel. He didn’t mind her way. It saved him the bother of ever making up his own mind about anything. Davy, do stop squirming like an eel.”

“I’ve nothing else to do,” protested Davy. “I can’t eat any more, and it’s no fun watching you and Anne eat.”

“Well, you and Dora go out and give the hens their wheat,” said Marilla. “And don’t you try to pull any more feathers out of the white rooster’s tail either.”

“I wanted some feathers for an Injun headdress,” said Davy sulkily. “Milty Boulter has a dandy one, made out of the feathers his mother give him when she killed their old white gobbler. You might let me have some. That rooster’s got ever so many more’n he wants.”

“You may have the old feather duster in the garret,” said Anne, “and I’ll dye them green and red and yellow for you.”

“You do spoil that boy dreadfully,” said Marilla, when Davy, with a radiant face, had followed prim Dora out. Marilla’s education had made great strides in the past six years; but she had not yet been able to rid herself of the idea that it was very bad for a child to have too many of its wishes indulged.

“All the boys of his class have Indian headdresses, and Davy wants one too,” said Anne. “I know how it feels . . . I’ll never forget how I used to long for puffed sleeves when all the other girls had them. And Davy isn’t being spoiled. He is improving every day. Think what a difference there is in him since he came here a year ago.”

“He certainly doesn’t get into as much mischief since he began to go to school,” acknowledged Marilla. “I suppose he works off the tendency with the other boys. But it’s a wonder to me we haven’t heard from Richard Keith before this. Never a word since last May.”

“I’ll be afraid to hear from him,” sighed Anne, beginning to clear away the dishes. “If a letter should come I’d dread opening it, for fear it would tell us to send the twins to him.”

A month later a letter did come. But it was not from Richard Keith. A friend of his wrote to say that Richard Keith had died of consumption a fortnight previously. The writer of the letter was the executor of his will and by that will the sum of two thousand dollars was left to Miss Marilla Cuthbert in trust for David and Dora Keith until they came of age or married. In the meantime the interest was to be used for their maintenance.

“It seems dreadful to be glad of anything in connection with a death,” said Anne soberly. “I’m sorry for poor Mr. Keith; but I am glad that we can keep the twins.”

“It’s a very good thing about the money,” said Marilla practically. “I wanted to keep them but I really didn’t see how I could afford to do it, especially when they grew older. The rent of the farm doesn’t do any more than keep the house and I was bound that not a cent of your money should be spent on them. You do far too much for them as it is. Dora didn’t need that new hat you bought her any more than a cat needs two tails. But now the way is made clear and they are provided for.”

Davy and Dora were delighted when they heard that they were to live at Green Gables, “for good.” The death of an uncle whom they had never seen could not weigh a moment in the balance against that. But Dora had one misgiving.

“Was Uncle Richard buried?” she whispered to Anne.

“Yes, dear, of course.”

“He . . . he . . . isn’t like Mirabel Cotton’s uncle, is he?” in a still more agitated whisper. “He won’t walk about houses after being buried, will he, Anne?”
"""

In [0]:
def clean_text_string(text):
    text = re.sub(r"[\"“”]", "", text)
    text = re.sub(r"[.!?]", "", text)
    text = re.sub(r";", "", text)
    text = re.sub(r"-", " ", text)

    # Remove whitespace (including newlines) from each line
    cleaned_lines = [line.strip() for line in text.splitlines()]
    cleaned_lines = [line for line in cleaned_lines if line]
    return cleaned_lines
    
reference_text = clean_text_string(reference_text)

reference_text = [re.sub(r"\s+", " ", line) for line in reference_text]
reference_text[1]

Out[6]: 'But Paul’s arms are longer’n mine, brumbled Davy They’ve had eleven years to grow and mine’ve only had seven ‘Sides, I did ask, but you and Anne was so busy talking you didn’t pay any ‘tention ‘Sides, Paul’s never been here to any meal escept tea, and it’s easier to be p’lite at tea than at breakfast You ain’t half as hungry It’s an awful long while between supper and breakfast Now, Anne, that spoonful ain’t any bigger than it was last year and I’m ever so much bigger'

Since the models will be trained on lowercase words only, it is important to ensure that the case of letters in the reference text is consistent. This ensures that words like "Honey" and "honey" are not treated as distinct by the model.

To address this, all letters in the `reference_text` will be converted to lowercase as part of the preprocessing step.


In [0]:
reference_text = [line.lower() for line in reference_text]

reference_text[1]

Out[7]: 'but paul’s arms are longer’n mine, brumbled davy they’ve had eleven years to grow and mine’ve only had seven ‘sides, i did ask, but you and anne was so busy talking you didn’t pay any ‘tention ‘sides, paul’s never been here to any meal escept tea, and it’s easier to be p’lite at tea than at breakfast you ain’t half as hungry it’s an awful long while between supper and breakfast now, anne, that spoonful ain’t any bigger than it was last year and i’m ever so much bigger'

## First Markov Chain

In general, a **First Markov Chain** is a model that predicts the next word in a sequence based solely on the current word.

##### Step 1: Extract Word Pairs
The first step involves defining a function called `extract_word_pairs`. This function processes the text and creates an RDD consisting of word pairs, where each word is paired with the word that follows it in the sequence. For example, from the text, we might see pairs like `(mrs, rachel)` or `(rachel, lynde)`.



In [0]:
# First: flatten the text into pairs of consecutive words
def extract_word_pairs(paragraph):
    list_words_of_paragraph = paragraph.lower().split()
    list_pairs_of_words_of_paragraph = []
    previous_word = ''
    for word in list_words_of_paragraph:
        if previous_word:
            list_pairs_of_words_of_paragraph.append((previous_word, word))
        previous_word = word
    return list_pairs_of_words_of_paragraph

text_pairs_of_words_rdd = cleaned_rdd.flatMap(extract_word_pairs)
text_pairs_of_words_rdd.take(10)


Out[8]: [('chapter', 'i'),
 ('mrs', 'rachel'),
 ('rachel', 'lynde'),
 ('lynde', 'is'),
 ('is', 'surprised'),
 ('mrs', 'rachel'),
 ('rachel', 'lynde'),
 ('lynde', 'lived'),
 ('lived', 'just'),
 ('just', 'where')]

#####Step 2: Group Word Pairs by Key
This step groups the word pairs by their **key** (the first word in the pair). For each key, we create a list of all the words that appeared after it in the text. 


In [0]:
#Pairs and list the possible next words
model_rdd = text_pairs_of_words_rdd.groupByKey().mapValues(list)
model_rdd.take(10)

Out[9]: [('chapter',
  ['i',
   'ii',
   'iii',
   'iv',
   'v',
   'vi',
   'vii',
   'viii',
   'ix',
   'x',
   'xi',
   'xii',
   'xiii',
   'xiv',
   'xv',
   'about,',
   'xvi',
   'xvii',
   'xviii',
   'xix',
   'xx',
   'xxi',
   'xxii',
   'xxiii',
   'xxiv',
   'xxv',
   'xxvi',
   'xxvii',
   'xxviii',
   'xxix',
   'xxx',
   'xxxi',
   'xxxii',
   'xxxiii',
   'xxxiv',
   'xxxv',
   'xxxvi',
   'xxxvii',
   'xxxviii',
   'i',
   'ii',
   'iii',
   'iv',
   'v',
   'vi',
   'vii',
   'viii',
   'ix',
   'x',
   'xi',
   'xii',
   'xiii',
   'xiv',
   'xv',
   'xvi',
   'xvii',
   'of',
   'xviii',
   'xix',
   'xx',
   'xxi',
   'xxii',
   'xxiii',
   'xxiv',
   'xxv',
   'xxvi',
   'xxvii',
   'xxviii',
   'xxix',
   'xxx',
   'of',
   'in',
   'did',
   'i',
   'ii',
   'iii',
   'iv',
   'v',
   'vi',
   'vii',
   'viii',
   'ix',
   'x',
   'xi',
   'xii',
   'xiii',
   'xiv',
   'xv',
   'xvi',
   'xvii',
   'xviii',
   'xix',
   'xx',
   'xxi',
   'xxii',
   'xxiii',


#####Step 3: Generate text using Markov Chain

In [0]:

# Generate text using first markov chain
def generate_text_rdd(model_rdd, start_word, num_words):
    markov_dict = model_rdd.collectAsMap()

    current_word = start_word
    generated_text = [current_word]

    for _ in range(num_words - 1): 
        if current_word in markov_dict:
            next_word = random.choice(markov_dict[current_word])  # Randomly select the next word
            generated_text.append(next_word)
            current_word = next_word  # Move to the next word
        else:
            break  # Stop if no next word is found

    return ' '.join(generated_text)

# Generate a story starting with a first word from the reference text
start_word = "so"
num_words_to_generate = 1606  # Length of the generated story - I want it to mach the length of it to the reference text, which is 1606 words

generated_story = generate_text_rdd(model_rdd, start_word, num_words_to_generate)

print(generated_story)

so tired out, and i will have thought out who had to get out to green gables in popular ladies were hovering anxiously to your strawberries we’ll be talking you were a dignity and mr harrison shook her she saw two heroines have his company but the americans from an avonlea folks were the main road sounds so you can it was shelling them out and dust to put them mary joe, up in vain about myself ginger’s profane habits but to it—just like minnie may’s life, said wickedly


In [0]:
# new

# Generate text using first markov chain
def generate_text_rdd(model_rdd, start_word, num_words):
    markov_dict = model_rdd.collectAsMap()

    current_word = start_word
    generated_text = [current_word]

    for _ in range(num_words - 1): 
        if current_word in markov_dict:
            next_word = random.choice(markov_dict[current_word])  # Randomly select the next word
           
        else:
            # choose next word from all dictionary
            next_word = random.choice(list(markov_dict))
            # next_word = random.choice(list(markov_dict.keys()))

        generated_text.append(next_word)
        current_word = next_word

    return ' '.join(generated_text)

# Generate a story starting with a first word from the reference text
start_word = "so"
num_words_to_generate = 1606  # Length of the generated story - I want it to mach the length of it to the reference text, which is 1606 words

generated_story = generate_text_rdd(model_rdd, start_word, num_words_to_generate)

print(generated_story)

so little patience with you—but, then, but he’s always warranted to copy you see, miss i feel queerer than a voice went away, hauling potatoes and there would never to spend our trinidad missionary, said aunt olivia was the name mrs rachel sent her some such a glassful of music, only youth but i’m wrong places march, said priscilla looked about alec and clearly, sweetly, over him st john’s bedroom, and corner by her very fond of exquisite grace of a funeral after them she told me by the commonplace as a twinkling, had fallen tree marilla i don’t believe that atoned for a nice dreams i’d never let the place suppose i’ll expect of temper at her ‘mother lavendar’ and captain who died just to make dora’s elbow it was impossible to mar it had held his farm, nor how to anyone, that i work cures like clumps of business to teach you are your eyes roam about the rules the haunted meadows between its ups and brother and an unholy shriek i’ve longed for a wonderful everything hurt awful to see hi

#####Step 4: Prepare data to calculate BLEU Score

I convert the reference data and genearted data to have the same format. To achieve that, I transform them into a lists of individual words by splitting the text on whitespace.


In [0]:
references = [word for line in reference_text for word in line.split()]
generated = generated_story.split()

#####Step 5: Calculate the BLEU Score

####What is BLEU Score?
According to [Google](https://cloud.google.com/translate/docs/advanced/automl-evaluate):

*BLEU (Bilingual Evaluation Understudy) score, which indicates how similar the candidate text is to the reference text. A BLEU score value that is closer to one indicates that a translation is closer to the reference text.*

------------------

In our scenario, we encountered the following error:  
**"The number of hypotheses and their reference(s) should be the same."**

This error occurs because the generated text is much shorter than the reference text. The reference text contains 1606 words, but the generated text is significantly shorter. Let’s check the word count of the generated text.

In [0]:
score = corpus_bleu(references, [generated])
print(score)

---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
File <command-4115528835488850>:1
----> 1 score = corpus_bleu(references, [generated])
      2 print(score)

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-afcca901-9341-48e4-b23d-d7d4b40f69ac/lib/python3.9/site-packages/nltk/translate/bleu_score.py:220, in corpus_bleu(list_of_references, hypotheses, weights, smoothing_function, auto_reweigh)
    217 p_denominators = Counter()  # Key = ngram order, and value = no. of ngram in ref.
    218 hyp_lengths, ref_lengths = 0, 0
--> 220 assert len(list_of_references) == len(hypotheses), (
    221     "The number of hypotheses and their reference(s) should be the " "same "
    222 )
    224 try:
    225     weights[0][0]

AssertionError: The number of hypotheses and their reference(s) should be the same 

In [0]:
print(f"Reference has {len(references)} words \nGenerated has {len(generated)} words")

Reference has 1606 words 
Generated has 89 words


This approach has a limitation due to its stop condition: if no next word is found, the text generation stops prematurely.

To address this, I extend the approach by introducing a fallback logic.

## First Markov Chain with Fallback Method

The key difference in this method lies in how the next word is chosen. If no valid next word is found in the model, a fallback word is used instead. For this implementation, I have manually defined the fallback word as `"the"`. This choice is based on its frequent usage in English, making it a reasonable and versatile default option.


In [0]:
def generate_text_rdd_with_fallback(model_rdd, start_word, num_words, fallback_word="the"):
    markov_dict = model_rdd.collectAsMap()

    current_word = start_word
    generated_text = [current_word]

    for _ in range(num_words - 1):  
        if current_word in markov_dict and markov_dict[current_word]:  # Check if there are valid next words
            next_word = random.choice(markov_dict[current_word])
        else:
            next_word = fallback_word  # Use the fallback word if no transitions are available

        generated_text.append(next_word)
        current_word = next_word

    return ' '.join(generated_text)

# Generate a story
start_word = "so"
num_words_to_generate = 1606

generated_story = generate_text_rdd_with_fallback(model_rdd, start_word, num_words_to_generate)
print(generated_story)


so much thinking of feeling to be out and thank you know i’ll go on it will hardly ever broken up yet full of that was having to quote it would be real diamond cluster of pearl beads compared their delicacy of his, and slipped it is not to help him if he had been in that matched her very important building of fear was listening to wait two off to the kitchen the roses of her lustrous in and fiercely, and then the wolf in his vaccination had been expecting some specious pretense but i don’t know, nodded about davy’s manners left out with scornful looking at this did anyone make funny aunt olivia she might be ashamed of barry’s nowhere having suffered by him and speech i couldn’t understand right had to have a very many, said miss cuthbert in a cry tommy sloane to hear about nine that you he knocked over there for leslie slowly, stiffly, falteringly, are themselves away safe and yet but please do it was coming out by yourself i like to look tired, and bright red ribbons and help you see,

In [0]:
references = [word for line in reference_text for word in line.split()]
generated = generated_story.split()

print(f"Reference has {len(references)} words \nGenerated has {len(generated)} words")

Reference has 1606 words 
Generated has 1606 words


In [0]:
# Evaluate BLEU score
references = [reference_text]
generated = generated_story.split()

score = corpus_bleu(references, [generated])
print("Generated BLEU score:", score)

#warning message:

Generated BLEU score: 8.17147163445869e-232


/local_disk0/.ephemeral_nfs/envs/pythonEnv-ddb53df3-afc7-40da-8053-d7b98533fdb8/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/local_disk0/.ephemeral_nfs/envs/pythonEnv-ddb53df3-afc7-40da-8053-d7b98533fdb8/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/local_disk0/.ephemeral_nfs/envs/pythonEnv-ddb53df3-afc7-40da-8053-d7b98533fdb8/lib/python3.9/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-

The calculated BLEU score is extremely low (`Generated BLEU score: 8.17147163445869e-232`), which is effectively close to zero.

This result indicates that the generated text has almost no overlap with the reference text, especially for higher-order n-grams.

---

#### **What are N-grams?**

N-grams are continuous sequences of words of length **N**, used to capture relationships between words in a sequence.

---

#### **Example**
Consider the sentence from our data:  
*"there are plenty of people in avonlea"*

- **Unigrams:**  
  `['there', 'are', 'plenty', 'of', 'people', 'in', 'avonlea']`
  
- **Bigrams:**  
  `['there are', 'are plenty', 'plenty of', 'of people', 'people in', 'in avonlea']`
  
- **Trigrams:**  
  `['there are plenty', 'are plenty of', 'plenty of people', 'of people in', 'people in avonlea']`

---

#### **Smoothing Function**
The warning message suggests applying a `SmoothingFunction` to address the lack of higher-order n-grams. This function prevents the BLEU score from dropping to zero by adjusting the penalization for missing higher n-grams, ensuring a more balanced evaluation of the generated text.
 


In [0]:
# Evaluate BLEU score
smoothing_function = SmoothingFunction().method1
score = corpus_bleu(references, [generated], smoothing_function=smoothing_function)

print("Generated BLEU score:", score)
# Generated BLEU score: 0.0003146941887548595


Generated BLEU score: 0.0003146941887548595


## Second-Order Markov Chain

A **Second-Order Markov Chain** is an extention of the First Order Markov Chain logic, by considering more context when predicting the next word. 

#### How It Differs from a First-Order Markov Chain
It predicts the next word based on the **current word and the word before it**.  

Advantages:
- it captures more context
- it reduces randomness in the text generation, because next possible words are narrowed

In [0]:
def function(paragraph):
    list_words = paragraph.lower().split()
    list_triples = []
    previous1 = ''
    previous2 = ''
    
    for item in list_words:
        if previous1 and previous2:
            list_triples.append((previous2, previous1, item))
        previous2 = previous1
        previous1 = item
        
    return list_triples

word_triples_rdd = cleaned_rdd.flatMap(function)

#Triples look like the following:
word_triples_rdd.take(10)

Out[20]: [('mrs', 'rachel', 'lynde'),
 ('rachel', 'lynde', 'is'),
 ('lynde', 'is', 'surprised'),
 ('mrs', 'rachel', 'lynde'),
 ('rachel', 'lynde', 'lived'),
 ('lynde', 'lived', 'just'),
 ('lived', 'just', 'where'),
 ('just', 'where', 'the'),
 ('where', 'the', 'avonlea'),
 ('the', 'avonlea', 'main')]

In [0]:

# Create the model
model_rdd = word_triples_rdd.map(lambda triple: ((triple[0], triple[1]), triple[2])).groupByKey().mapValues(list)

model_rdd.take(6)


Out[21]: [(('rachel', 'lynde'),
  ['is',
   'lived',
   'was',
   'did',
   'is',
   'got',
   'are',
   'had',
   'was',
   'for',
   'said',
   'had',
   'was',
   'was',
   'drove',
   'would',
   'came',
   'to',
   'she’s',
   'beats',
   'if',
   'would',
   'would',
   'said',
   'scornfully',
   'would',
   'since',
   'said',
   'was',
   'when',
   'would',
   'afterwards',
   'have',
   'and',
   'sent',
   'as']),
 (('lynde', 'is'),
  ['surprised',
   'properly',
   'away',
   'a',
   'a',
   'feeling',
   'a',
   'gone',
   'a',
   'a—']),
 (('lynde', 'lived'), ['just']),
 (('lived', 'just'), ['where']),
 (('just', 'where'),
  ['the', 'the', 'the', 'the', 'we', 'her', 'the', 'the', 'he']),
 (('road', 'dipped'), ['down'])]

Since the Second-Order Markov Chain takes two words into consideration when predicting the next word, we can initialize the text generation process by providing the first two words from the reference text. 


In [0]:
def generate_text_trigram(model_rdd, start_word1, start_word2, num_words):
    current_word1 = start_word1
    current_word2 = start_word2
    generated_text = [current_word1, current_word2]
    
    for _ in range(num_words - 2):  # (We already have the first two words)
        # Get the list of possible next words for the current two words
        next_word_rdd = model_rdd.filter(lambda pair: pair[0] == (current_word1, current_word2)) \
                                  .map(lambda filtered_pair: random.choice(filtered_pair[1]))
        
        next_word_list = next_word_rdd.take(1)
        
        if next_word_list:
            next_word = next_word_list[0]
            generated_text.append(next_word)
            current_word1, current_word2 = current_word2, next_word 
        else:
            break  

    return ' '.join(generated_text)

start_word1 = "so"  # First word
start_word2 = "you"
num_words_to_generate = 1606

generated_story_trigram = generate_text_trigram(model_rdd, start_word1, start_word2, num_words_to_generate)

print(generated_story_trigram)

so you just accept philippa gordon, greeting


##Fallback Approach on Second-Order Markov Chain

In the generated output above, we observe that the story is very short. This happens because the model stops generating text whenever it fails to find a valid next word for a given sequence. To adress this limitation, once again, I implement a fallback mechanism.


In [0]:
def generate_text_trigram_with_fallback(model_rdd, start_word1, start_word2, num_words, fallback_word="the"):
    markov_dict = model_rdd.collectAsMap()

    current_word1 = start_word1
    current_word2 = start_word2
    generated_text = [current_word1, current_word2]

    for _ in range(num_words - 2):  
        if (current_word1, current_word2) in markov_dict and markov_dict[(current_word1, current_word2)]:
            next_word = random.choice(markov_dict[(current_word1, current_word2)])
        else:
            next_word = fallback_word  # Use the fallback word if no transitions are available

        generated_text.append(next_word)
        current_word1, current_word2 = current_word2, next_word  # Move to the next pair

    return ' '.join(generated_text)

start_word1 = "so"  
start_word2 = "you"
num_words_to_generate = 1606

generated_story = generate_text_trigram_with_fallback(model_rdd, start_word1, start_word2, num_words_to_generate)
print(generated_story)


so you may do for a small bottle partially filled with the over harbor people, laughed and subsided into silence over a farmers’ advocate on the occasion and showed no desire whatever to publish abroad the next door came over to see leslie before i opened it was hardly a spellbound princess, laughed diana i don’t know what i said, but something always occurred to her hair in the sitting room she had dreamed they will spend the time you don’t come half often enough on my head against your knee that’s the first time he comes and i have two delightful old china had explored spencervale, and, getting word of judgment and comparison not that i made allowances for her will be left at the open door to see them as a fat woman who has been it’s been preying on my lap i won’t be the neighbor whom anne had made a pudding that was unfortunately true it seems that he is my dress is a reduced gentlewoman, explained miss barry gave diana currant wine instead of wakening him up in the parlor is tiny a

In [0]:
references = [word for line in reference_text for word in line.split()]
generated = generated_story.split()

print(f"Reference has {len(references)} words \nGenerated has {len(generated)} words")

Reference has 1606 words 
Generated has 1606 words


the length of the generated and referenc text is the same, so I calculate BLEU score with the smoothing function. the smoothng function can be calculate using several methods. 

In [0]:
# Evaluate BLEU score - NO SMOOTHING FUNCTION
references = [reference_text]
generated = generated_story.split()

score = corpus_bleu(references, [generated])
print("Generated BLEU score:", score)

Generated BLEU score: 8.17147163445869e-232


`Generated BLEU score:  8.17147163445869e-232`

This is super low.
Apply smoothing function. 

method1 helps avoid zero counts for n-grams that are missing in the candidate translation but appear in the reference translations.

`Generated BLEU score: 0.0003146941887548595` - better result


method2 uses a different smoothing strategy, which can involve adjusting the contribution of higher-order n-grams more effectively. In particular, it adjusts the penalty that would be applied for missing higher-order n-grams.  Good for cases where you expect fewer higher-order n-gram matches.

`Generated BLEU score: 0.0017688284648976552` - even better

In [0]:
references = [reference_text]
generated = generated_story.split()

# Evaluate BLEU score
smoothing_function = SmoothingFunction().method1
score = corpus_bleu(references, [generated], smoothing_function=smoothing_function)
print("Generated BLEU score:", score)

Generated BLEU score: 0.0003146941887548595


In [0]:
references = [reference_text]
generated = generated_story.split()

# Evaluate BLEU score
smoothing_function = SmoothingFunction().method2
score = corpus_bleu(references, [generated], smoothing_function=smoothing_function)
print("Generated BLEU score:", score)

Generated BLEU score: 0.0017688284648976552


##Weighted Random Selection: Incorporating weighted probabilities, with fallbacks to bigrams and unigrams.

In the following approach, weighted random selection is used to choose the next word in the generated text.

####Weighted Random Selection
The weight signifies the likelihood or probability of selecting that word.

##### How Weights Are Calculated:
For each trigram, bigram, or unigram, the weight of a word is determined by its frequency. Specifically:
- In a trigram, the weight of a word is the count of its occurrences following the given word pair. For example, if `"I like"` is followed by `"apples"` 5 times in the text, `"apples"` will have a weight of 5.
- This principle also applies to bigrams and unigrams, where the weight corresponds to the frequency of the word's occurrence.


Words that appear more frequently with a given pair (in trigrams) or word (in unigrams) will have higher weights, making them more likely to be chosen.

**Fallback:**

If the trigram pair is not found, the same logic is applied to bigrams.
When falling back to unigrams, weights are directly based on word frequencies.

In [0]:
# Function to extract trigrams
def function(paragraph):
    list_words = paragraph.lower().split()
    list_triples = []
    previous1 = ''
    previous2 = ''
    
    for item in list_words:
        if previous1 and previous2:
            list_triples.append((previous2, previous1, item))
        previous2 = previous1
        previous1 = item
        
    return list_triples

# Apply function to create trigrams
word_triples_rdd = cleaned_rdd.flatMap(function)

# Create trigram model
model_rdd = word_triples_rdd.map(lambda triple: ((triple[0], triple[1]), triple[2])).groupByKey().mapValues(list)

# Create bigram model
bigram_rdd = word_triples_rdd.map(lambda triple: (triple[:2], triple[2])).groupByKey().mapValues(list)

# Create unigram model
unigram_rdd = cleaned_rdd.flatMap(lambda x: x.lower().split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

import random

def weighted_choice(choices):
    words, weights = zip(*choices)
    return random.choices(words, weights=weights, k=1)[0]


def generate_text_trigram_with_fallback(model_rdd, bigram_rdd, unigram_rdd, start_word1, start_word2, num_words, fallback_word="the"):
    markov_dict = model_rdd.collectAsMap()
    bigram_dict = bigram_rdd.collectAsMap()
    unigram_dict = unigram_rdd.collectAsMap()
    
    # Prepare unigram weights for fallback
    unigram_weights = [(word, count) for word, count in unigram_dict.items()]
    
    current_word1 = start_word1
    current_word2 = start_word2
    generated_text = [current_word1, current_word2]

    for _ in range(num_words - 2):
        if (current_word1, current_word2) in markov_dict and markov_dict[(current_word1, current_word2)]:
            next_word = weighted_choice([(word, markov_dict[(current_word1, current_word2)].count(word)) 
                                         for word in markov_dict[(current_word1, current_word2)]])
        elif (current_word1, current_word2) in bigram_dict and bigram_dict[(current_word1, current_word2)]:
            next_word = weighted_choice([(word, bigram_dict[(current_word1, current_word2)].count(word)) 
                                         for word in bigram_dict[(current_word1, current_word2)]])
        elif unigram_weights:
            next_word = weighted_choice(unigram_weights)
        else:
            next_word = fallback_word  # Use the fallback word if no transitions are available

        generated_text.append(next_word)
        current_word1, current_word2 = current_word2, next_word  # Move to the next pair

    return ' '.join(generated_text)


start_word1 = "so"
start_word2 = "you"
num_words_to_generate = 1606

generated_story = generate_text_trigram_with_fallback(model_rdd, bigram_rdd, unigram_rdd, start_word1, start_word2, num_words_to_generate)
generated_story


Out[8]: "so you see mr harrison was really out of it but ranks of spruce the tinkles of sleigh bells and distant laughter, that seemed a few moments of it, said anne seriously i shall never forgive jane for not coming back to this astonishing change pupil still tablecloth sake put in the parlor, where a ladder up to the very thought of which he was a very different from avonlea the others what her feelings hurt it can’t be i want to do it of course, i had to stop quick, she’s better off as viciously as if, somehow, diana had ever been known to the house of dreams and grieves and rejoices and lives becomes inseparably connected with avonlea, and as she was a very wild set of boys, who were carrying gilbert blythe has to eat i know that it was a little boy and when i was reading it at all i know, said captain jim was the rev mr allan had finished his devotions he leaned back in the world and not just as soon as i would understand can’t you walk down to the island the last time i’ll be as

In [0]:
# Evaluate BLEU score
references = [reference_text]
generated = generated_story.split()

smoothing_function = SmoothingFunction().method2
score = corpus_bleu(references, [generated], smoothing_function=smoothing_function)
print("Generated BLEU score:", score)

Generated BLEU score: 0.001907998581067857


`Generated BLEU score: 0.001907998581067857` is the best result so far. 

One last adjustment will be done. 

##Final Version

**Counter**:  This time I use Python's `Counter` to precompute and store word frequencies, which is better for large datasets. 

**Fallback to reference text**: If neither trigram nor bigram predictions are available, the model uses words from the reference text as a fallback. This allows the generated text remains stylistically similar by using simmilar words to the reference material.


-----------

####The final BLEU Score is:
`Generated BLEU score: 0.0020444282652433286` - which is the best result at this point.


In [0]:
from collections import Counter

# Create trigram model with precomputed weights
model_rdd = word_triples_rdd.map(lambda triple: ((triple[0], triple[1]), triple[2])) \
                            .groupByKey() \
                            .mapValues(lambda words: Counter(words))

# Create bigram model with precomputed weights
bigram_rdd = word_triples_rdd.map(lambda triple: (triple[:2], triple[2])) \
                             .groupByKey() \
                             .mapValues(lambda words: Counter(words))

# Function for weighted random selection
def weighted_choice(weighted_dict):
    words, weights = zip(*weighted_dict.items())
    return random.choices(words, weights=weights, k=1)[0]

# Function to generate text with fallback
def generate_text_trigram_with_fallback(model_rdd, bigram_rdd, references, start_word1, start_word2, num_words):
    markov_dict = model_rdd.collectAsMap()  # Trigram dictionary with weights
    bigram_dict = bigram_rdd.collectAsMap()  # Bigram dictionary with weights

    reference_weights = Counter(references)

    current_word1 = start_word1
    current_word2 = start_word2
    generated_text = [current_word1, current_word2]

    for _ in range(num_words - 2):
        # Trigram case
        if (current_word1, current_word2) in markov_dict and markov_dict[(current_word1, current_word2)]:
            next_word = weighted_choice(markov_dict[(current_word1, current_word2)])
        # Bigram fallback
        elif (current_word1, current_word2) in bigram_dict and bigram_dict[(current_word1, current_word2)]:
            next_word = weighted_choice(bigram_dict[(current_word1, current_word2)])
        # Reference text fallback
        else:
            next_word = weighted_choice(reference_weights)

        generated_text.append(next_word)
        current_word1, current_word2 = current_word2, next_word  # Move to the next pair

    return ' '.join(generated_text)

start_word1 = "so"
start_word2 = "you"
num_words_to_generate = 1606

generated_story = generate_text_trigram_with_fallback(model_rdd, bigram_rdd, references, start_word1, start_word2, num_words_to_generate)

generated_story


Out[19]: "so you may be all right—i’m not frivolous at heart she’s young and fair in silk attire, a short, stout gray haired lady in pink silk waist with a strange solemnity about the pudding sauce again and i had heard lately from gilbert, but passed her hard work to use a slang term, have any spare saturday afternoons at the foot, and as usual that night of their first approach to a girl once—went to school the first few weeks i wouldn’t likely find married life as dick’s wife i reckon i’m thankful she did—i prayed that stephen irving paul’s father stephen irving came forward beamingly, with outstretched hand and dragging me along with his head his face somehow, anne, i just rushed over like this and i saved his life, was it you must be even better some day if i have been read at your house come on friday, said marilla crossly yes, you can eat any hour and which the old lady sadly but you can go with her, but it looks to the square, deep set window from out the avis last friday morni

In [0]:
# Evaluate BLEU score
references = [reference_text]
generated = generated_story.split()

smoothing_function = SmoothingFunction().method2
score = corpus_bleu(references, [generated], smoothing_function=smoothing_function)
print("Generated BLEU score:", score)

Generated BLEU score: 0.0020444282652433286


**Aleksandra Graczyk**

SGH

Advanced Analytics - Big Data